In [2]:
import torch
import triton
import triton.language as tl

DEVICE = triton.runtime.driver.active.get_active_torch_device()


def is_hip():
    return triton.runtime.driver.active.get_current_target().backend == "hip"


def is_cuda():
    return triton.runtime.driver.active.get_current_target().backend == "cuda"


def supports_host_descriptor():
    return is_cuda() and torch.cuda.get_device_capability()[0] >= 9


def is_blackwell():
    return is_cuda() and torch.cuda.get_device_capability()[0] == 10


def is_hopper():
    return is_cuda() and torch.cuda.get_device_capability()[0] == 9

if is_hip():
    NUM_STAGES_OPTIONS = [1]
elif supports_host_descriptor():
    NUM_STAGES_OPTIONS = [2, 3, 4]
else:
    NUM_STAGES_OPTIONS = [2, 3, 4]

def _host_descriptor_pre_hook(nargs):
    BLOCK_M = nargs["BLOCK_M"]
    BLOCK_N = nargs["BLOCK_N"]
    HEAD_DIM = nargs["HEAD_DIM"]
    if not isinstance(nargs["desc_q"], tl.tensor_descriptor):
        return
    nargs["desc_q"].block_shape = [BLOCK_M, HEAD_DIM]
    if nargs["FP8_OUTPUT"]:
        nargs["desc_v"].block_shape = [HEAD_DIM, BLOCK_N]
    else:
        nargs["desc_v"].block_shape = [BLOCK_N, HEAD_DIM]
    nargs["desc_k"].block_shape = [BLOCK_N, HEAD_DIM]
    nargs["desc_o"].block_shape = [BLOCK_M, HEAD_DIM]
    
configs = [
    triton.Config({'BLOCK_M': BM, 'BLOCK_N': BN}, num_stages=s, num_warps=w, pre_hook=_host_descriptor_pre_hook) \
    for BM in [64, 128]\
    for BN in [32, 64, 128]\
    for s in NUM_STAGES_OPTIONS \
    for w in [4, 8]\
]

def keep(conf):
    BLOCK_M = conf.kwargs["BLOCK_M"]
    BLOCK_N = conf.kwargs["BLOCK_N"]
    return not (is_cuda() and torch.cuda.get_device_capability()[0] == 9 and BLOCK_M * BLOCK_N < 128 * 128
                and conf.num_warps == 8)
    
def prune_invalid_configs(configs, named_args, **kwargs):
    S = kwargs["S"]

    # Filter out configs where BLOCK_M > N_CTX
    return [conf for conf in configs if conf.kwargs.get("BLOCK_M", 0) <= S]

In [ ]:
@triton.jit
def _maybe_make_tensor_desc(desc_or_ptr, shape, strides, block_shape):
    if isinstance(desc_or_ptr, tl.tensor_descriptor):
        return desc_or_ptr
    else:
        return tl.make_tensor_descriptor(desc_or_ptr, shape, strides, block_shape)


@triton.autotune(configs=list(filter(keep, configs)), key=["S", "HEAD_DIM", "FP8_OUTPUT", "warp_specialize"],
                 prune_configs_by={'early_config_prune': prune_invalid_configs})
@triton.jit
def _attn_fwd_kernel(
    sm_scale,
    M,
    Bs,
    NH,
    q_desc,
    k_desc,
    v_desc,
    o_desc,
    S,
    HEAD_DIM:tl.constexpr,
    BLOCK_M:tl.constexpr,
    BLOCK_N:tl.constexpr,
    FP8_OUTPUT:tl.constexpr,
    STAGE:tl.constexpr,
    warp_specialize:tl.constexpr,
    IS_HOPPER:tl.constexpr,
):
    dtype = tl.float8e5 if FP8_OUTPUT else tl.float16
    tl.static_assert(BLOCK_N <= HEAD_DIM)
    
    

In [ ]:
class _attention(torch.autograd.Function):
    @staticmethod
    def forward(ctx, q: torch.Tensor, k: torch.Tensor, v: torch.Tensor, causal: bool, sm_scale: float, warp_specialize: bool = True):
        """
        q.shape: (Bs, NHead, SeqLen, HeadDim)
        """
        assert q.dim() == 4
        Bs, NH, S, HEAD_DIM = q.shape
        HEAD_DIM_K, HEAD_DIM_V = k.shape[-1], v.shape[-1]
        assert HEAD_DIM == HEAD_DIM_K == HEAD_DIM_V
        assert HEAD_DIM_K in {16, 32, 64, 128, 256}

        o = torch.empty_like(q)
        stage = 3 if causal else 1

        M = torch.empty((Bs, NH, S), device=q.device, dtype=torch.float32)
        if supports_host_descriptor() and not (is_hopper() and warp_specialize):
            y_dim = Bs * NH * S
            dummy_shape = [1, 1]
            q_desc = tl.make_tensor_descriptor(q, shape=[y_dim, HEAD_DIM], strides=[y_dim, 1], block_shape=dummy_shape)
            k_desc = tl.make_tensor_descriptor(k, shape=[y_dim, HEAD_DIM], strides=[y_dim, 1], block_shape=dummy_shape)
            o_desc = tl.make_tensor_descriptor(o, shape=[y_dim, HEAD_DIM], strides=[y_dim, 1], block_shape=dummy_shape)
            if q.dtype == torch.float8_e5m2:
                v_desc = tl.make_tensor_descriptor(v, shape=[HEAD_DIM, y_dim], strides=[S, 1], block_shape=dummy_shape)
            else:
                v_desc = tl.make_tensor_descriptor(v, shape=[y_dim, HEAD_DIM], strides=[y_dim, 1], block_shape=dummy_shape)
        else:
            q_desc = q
            k_desc = k
            v_desc = v
            o_desc = o

        triton.set_allocator(lambda size, align, _: torch.empty(size, dtype=torch.int8, device="cuda"))
        ctx.grid = lambda META: (triton.cdiv(S, META["BLOCK_M"]), Bs * NH, 1)
        _attn_fwd_kernel[ctx.grid](
            sm_scale,
            M,
            Bs,
            NH,
            q_desc,
            k_desc,
            v_desc,
            o_desc,
            S=S,
            HEAD_DIM=HEAD_DIM,
            FP8_OUTPUT=q.dtype == torch.float8_e5m2,
            STAGE=stage,
            warp_specialize=warp_specialize,  #
            IS_HOPPER=is_hopper(),  #
            **extra_kern_args
        )
        ctx.save_for_backward(q, k, v, o, M)
        ctx.sm_scale = sm_scale
        ctx.HEAD_DIM = HEAD_DIM
        ctx.causal = causal
        return o